In [1]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
from shapely.geometry import shape as s_shape

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

In [2]:
path = DATA_DIR/'cairns_gtfs.zip'
feed = gt.read_gtfs(path, dist_units_out='km')

# Pick date
date = gt.get_first_week(feed)[0]
print('date', date)


date 20140526


In [4]:
tid = feed.trips['trip_id'].iat[0]
gt.build_trip_geojson(feed, tid, include_stops=True)

{'features': [{'geometry': {'coordinates': ((145.664847, -16.74631),
     (145.667393, -16.744308),
     (145.66825500000002, -16.743632),
     (145.66825500000002, -16.743632),
     (145.668394, -16.743522),
     (145.671277, -16.741258),
     (145.671074, -16.744007),
     (145.671074, -16.744015),
     (145.671074, -16.744015),
     (145.67106, -16.744732),
     (145.670999, -16.747083),
     (145.671118, -16.749574),
     (145.66965, -16.749645),
     (145.668625, -16.749381),
     (145.66853600000002, -16.749351),
     (145.667907, -16.749206),
     (145.667907, -16.749206),
     (145.667705, -16.749159),
     (145.667483, -16.749084),
     (145.66547, -16.748589000000003),
     (145.664198, -16.748260000000002),
     (145.663698, -16.74813),
     (145.663698, -16.74813),
     (145.662848, -16.74791),
     (145.662529, -16.749199),
     (145.662112, -16.751266),
     (145.66206499999998, -16.751911),
     (145.661993, -16.752660000000002),
     (145.662026, -16.753497),
     (145.

In [ ]:
route_id = feed.routes['route_id'].iat[0]
geo = gt.build_route_geojson(feed, route_id, include_stops=True)
path = DATA_DIR/'route_{!s}.geojson'.format(route_id)
with path.open('w') as tgt:
    json.dump(geo, tgt)

In [ ]:
# Add distances to feed
trips_stats = gt.compute_trips_stats(feed, compute_dist_from_shapes=True)
feed.stop_times = gt.add_dist_to_stop_times(feed, trips_stats)
feed.stop_times.T

In [ ]:
trips_stats = gt.compute_trips_stats(feed)
routes_stats = gt.compute_routes_stats(feed, trips_stats, date)
routes_stats


In [ ]:
a = pd.Series([np.nan, np.nan])
a.dropna().max()
#gt.get_trips(feed, date, "07:30:00")


In [ ]:
st['departure_time'].unique

In [ ]:
tl = gt.compute_trips_locations(feed, date, ['12:00:00'])
tl

In [ ]:
freq = '12H'
f = gt.compute_routes_time_series(feed, trips_stats, date, split_directions=True, freq=freq)

# cols_and_hows = [
#     ('num_trips', 'mean'),
#     ('num_trip_starts', 'sum'),
#     ('service_distance', 'sum'),
#     ('service_duration', 'sum'),
#     ]

# frames = []
# for col, how in cols_and_hows:
#     frames.append(f[col].resample(freq).agg({col: how}))    
# g = pd.concat(frames, axis=1)
# g = g.sortlevel(axis=1)

# #f = f.resample(freq).sum()

# g.columns.names = f.columns.names

# g.T
f.T

In [ ]:

# Pick screen line
path = DATA_DIR/'cairns_screen_line.geojson'
with path.open() as src:
    line = json.load(src)
    line = s_shape(line['features'][0]['geometry'])

g = gt.compute_screen_line_counts(feed, line, date)
g

In [ ]:
g.groupby(['route_id', 'orientation']).count()